In [1]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00


In [17]:
#@title Import necessary libraries and functions
import numpy as np
import pandas as pd
import matplotlib as pyplot
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from collections import Counter
import string
import re
from keras.preprocessing.text import Tokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from string import punctuation
from datasets import load_dataset
import transformers as ppb

In [9]:
from tqdm import tqdm
import pandas as pd
from string import punctuation

In [3]:
from google.colab import drive
drive.mount("/content/googledrive",force_remount=True)

Mounted at /content/googledrive


In [12]:
df=pd.read_csv("/content/googledrive/MyDrive/IMDB Dataset.csv")

In [13]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [14]:
df['review'] = df['review'].str.lower()
df['review'] = df['review'].apply(lambda x: ''.join([c for c in x if c not in punctuation]))

# Split by new lines and spaces
df['review'] = df['review'].str.replace('\n', ' ')
df['review'] =df['review'].str.split()

# Create a list of words
df['review'] = df['review'].apply(lambda x: ' '.join(x))

In [15]:
#@title Get the dataset
df['sentiment'] = [1 if sentiment == 'positive' else 0 for sentiment in df['sentiment']]
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production br br the filmin...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1


In [18]:
#@title Get the pretrained BERT model
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [19]:
#@title Split data into train, val and test sets
df=df[:10000]
xtrainval,xtest,ytrainval,ytest = train_test_split(df['review'], df['sentiment'], test_size=0.33, random_state=42)
xtrain,xval,ytrain,yval = train_test_split(xtrainval,ytrainval, test_size=0.2,random_state=42)


In [20]:
#@title Encode Data
encoded_data_train = tokenizer.batch_encode_plus(
    xtrain,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)
encoded_data_val = tokenizer.batch_encode_plus(
    xval,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
token_type_ids_train = encoded_data_train['token_type_ids']
labels_train = torch.tensor(ytrain.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
token_type_ids_val = encoded_data_val['token_type_ids']
labels_val = torch.tensor(yval.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [21]:
#@title Create Datasests and Dataloaders
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,
                              token_type_ids_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val,
                            token_type_ids_val,
                            labels_val)

batch_size = 4

train_loader = DataLoader(dataset_train, shuffle=True, batch_size=batch_size,  drop_last=True)
valid_loader = DataLoader(dataset_val, shuffle=False, batch_size=batch_size, drop_last=True)

In [22]:
#@title Customizing BERT for sentiment analysis
class CustomBERT(nn.Module):
  def __init__(self):
    super(CustomBERT,self).__init__()
    self.bert = model
    self.linear = nn.Linear(768,1)

  def freeze_bert(self):
    for param in self.bert.parameters():
      param.requres_grad = False

  def forward(self,inputs):
    #encoded_review = tokenizer(input_review,return_tensors="pt")
    bert_embeddings=self.bert(**inputs)
    #print(bert_embeddings['pooler_output'].shape)
    cls_embedding = bert_embeddings['pooler_output'] ## first token in the hidden state
    linear_op = self.linear(cls_embedding)

    return linear_op

In [23]:
#@title Set the device
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

In [24]:
#@title Define your model
#@title Define model parameters
SentimentBERT = CustomBERT()
SentimentBERT = SentimentBERT.to(device)
#print(SentimentBERT)
## learning rate
lr=0.001

## loss function - binary cross entropy loss as derived
SentimentBERT.freeze_bert()

In [25]:
#@title Define training function
def train_model(train_dataloader,val_dataloader,epochs=3,lr=0.001):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(SentimentBERT.parameters(), lr=lr)

    best_val = torch.inf

    for epoch in range(epochs):

      ## Training
      SentimentBERT.train()
      train_loss = 0
      correct = 0
      total = 0
      for batch in tqdm(train_dataloader,desc="Batch training"):
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'token_type_ids': batch[2]
        }
        labels = batch[3]
        optimizer.zero_grad()
        outputs = SentimentBERT.forward(inputs)
        #print('batch outputs:',outputs)
        #print('batch labels:', labels)
        loss = criterion(outputs.squeeze(),labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(SentimentBERT.parameters(), max_norm=1.0)
        optimizer.step()
        train_loss += loss.item()
        predicted = torch.round(nn.functional.sigmoid(outputs).squeeze())
        #print('predicted:',predicted)
        correct += (predicted == labels.float()).sum()
        #print(f'Training Accuracy for batch:',correct/total)
        #print('correct:',correct)
        total += len(labels)

      print(f'Training Accuracy for epoch{epoch}:',correct/total)

      ## Validation
      SentimentBERT.eval()
      val_loss = 0
      correct = 0
      total = 0
      with torch.no_grad():
        for batch in val_dataloader:
          batch = tuple(b.to(device) for b in batch)
          inputs = {
              'input_ids': batch[0],
              'attention_mask': batch[1],
              'token_type_ids': batch[2]
          }
          labels = batch[3]
          outputs = SentimentBERT.forward(inputs)
          loss = criterion(outputs.squeeze(),labels.float())
          val_loss += loss.item()
          predicted = torch.round(nn.functional.sigmoid(outputs).squeeze())
          correct += (predicted == labels.float()).sum()
          total += len(labels)

      ## view results
      print(f'Validation Accuracy for epoch{epoch}:',correct/total)
      avg_train_loss = train_loss / len(train_dataloader)
      avg_val_loss = val_loss / len(val_dataloader)

      if avg_val_loss < best_val:
        best_val = avg_val_loss
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}, Model saved")
        torch.save(SentimentBERT.state_dict(),'best_model.pth')

In [26]:
train_model(train_loader,valid_loader,epochs=3,lr=0.00002)

Batch training: 100%|██████████| 1340/1340 [04:38<00:00,  4.81it/s]


Training Accuracy for epoch0: tensor(0.8394, device='cuda:0')
Validation Accuracy for epoch0: tensor(0.8604, device='cuda:0')
Epoch 1/3, Train Loss: 0.47036109831576833, Val Loss: 0.5432828593067824, Model saved


Batch training: 100%|██████████| 1340/1340 [04:42<00:00,  4.75it/s]


Training Accuracy for epoch1: tensor(0.9347, device='cuda:0')
Validation Accuracy for epoch1: tensor(0.8925, device='cuda:0')
Epoch 2/3, Train Loss: 0.2784632987884019, Val Loss: 0.3531568190374815, Model saved


Batch training: 100%|██████████| 1340/1340 [04:41<00:00,  4.75it/s]


Training Accuracy for epoch2: tensor(0.9737, device='cuda:0')
Validation Accuracy for epoch2: tensor(0.8821, device='cuda:0')


In [27]:
#@title Encode test data
encoded_data_test = tokenizer.batch_encode_plus(
    xtest,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
token_type_ids_test = encoded_data_test['token_type_ids']
labels_test = torch.tensor(ytest.values)

dataset_test = TensorDataset(input_ids_test,
                            attention_masks_test,
                            token_type_ids_test,
                            labels_test)

batch_size = 4
test_loader = DataLoader(dataset_test, shuffle=False, batch_size=batch_size,  drop_last=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [28]:
def evaluate_model(test_loader):
  SentimentBERT.eval()
  criterion = nn.BCEWithLogitsLoss()

  with torch.no_grad():
    SentimentBERT.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
      for batch in test_loader:
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'token_type_ids': batch[2]
        }
        labels = batch[3]
        outputs = SentimentBERT.forward(inputs)
        loss = criterion(outputs.squeeze(),labels.float())
        val_loss += loss.item()
        predicted = torch.round(nn.functional.sigmoid(outputs).squeeze())
        correct += (predicted == labels.float()).sum()
        total += len(labels)

    print('Test Accuracy :',correct/total)

In [29]:
SentimentBERT = CustomBERT()
SentimentBERT.load_state_dict(torch.load('best_model.pth'))
SentimentBERT = SentimentBERT.to(device)

In [30]:
 evaluate_model(test_loader)

Test Accuracy : tensor(0.9079, device='cuda:0')


## Without freezing parameters

In [31]:
#@title Define your model
#@title Define model parameters
SentimentBERT = CustomBERT()
SentimentBERT = SentimentBERT.to(device)
#print(SentimentBERT)
## learning rate
lr=0.001

## loss function - binary cross entropy loss as derived
#SentimentBERT.freeze_bert()

In [32]:
train_model(train_loader,valid_loader,epochs=3,lr=0.00002)

Batch training: 100%|██████████| 1340/1340 [04:41<00:00,  4.75it/s]


Training Accuracy for epoch0: tensor(0.9631, device='cuda:0')
Validation Accuracy for epoch0: tensor(0.8888, device='cuda:0')
Epoch 1/3, Train Loss: 0.16459965036900492, Val Loss: 0.6934061756950921, Model saved


Batch training: 100%|██████████| 1340/1340 [04:42<00:00,  4.75it/s]


Training Accuracy for epoch1: tensor(0.9806, device='cuda:0')
Validation Accuracy for epoch1: tensor(0.8993, device='cuda:0')
Epoch 2/3, Train Loss: 0.0999895663387355, Val Loss: 0.630177377954598, Model saved


Batch training: 100%|██████████| 1340/1340 [04:41<00:00,  4.75it/s]


Training Accuracy for epoch2: tensor(0.9901, device='cuda:0')
Validation Accuracy for epoch2: tensor(0.8948, device='cuda:0')


In [33]:
SentimentBERT = CustomBERT()
SentimentBERT.load_state_dict(torch.load('best_model.pth'))
SentimentBERT = SentimentBERT.to(device)

In [34]:
evaluate_model(test_loader)

Test Accuracy : tensor(0.9042, device='cuda:0')
